# Performance by product --> SoM Dashboard

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/EF/Corner/Brands')
FILE_NAME = 'Nestle_marcas.csv'

## Import

In [2]:
import pandas as pd

df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME))
df.columns = ['categoria','comp','nestle']
df.set_index('categoria', inplace=True)
df.sample(2)

,comp,nestle
categoria,,
Modificadores de Leche,ChocoMilk; Cal-c-tose; Avena Rivero Chocolate;...,"Carlos V polvo, Nesquik, Morelia, Abuelita polvo"
Chocolate Mesa,"Don GUSTAVO, Selecto; La Suiza Chocolate Mesa;...",Abuelita


## Transform

In [3]:
total = pd.DataFrame()

for col in df.columns:
    df[col] = df[col].str.replace(',',';')
    aux = df[col].str.split(';', expand=True).melt(ignore_index=False).dropna().iloc[:,-1:].rename({'value':'brand'},axis=1)
    aux['CPG'] = col.title()
    total = total.append(aux, ignore_index=False)

total['brand'] = total['brand'].str.strip()
total.sample(2)

,brand,CPG
categoria,,
Agua Saborizada,Be Light,Comp
Alimentos Infantiles,Parent's Choice,Comp


## Export

In [4]:
total.to_csv(BASE_DIR.joinpath(''.join(FILE_NAME.split('.')[:-1])+'_correct.csv'), sep=',', encoding='utf-16')